In [ ]:
import json

In [ ]:
class Method:
    def __init__(self, ID, name):
        self.ID = ID
        self.name = name
        self.nCalled = 0
        self.durations = []
        self.calls_to = {}

    def add_call(self, to_method):
        if to_method.ID in self.calls_to:
            self.calls_to[to_method.ID].nCalled += 1
        else:
            to_method.nCalled = 1
            self.calls_to[to_method.ID] = to_method
            
    def micros(self):
        return sum(self.durations)
      
    def millis(self):
        return int(self.micros() / 1000)

    def __str__(self):
        return f"{self.name.split(".")[-1]} (~{self.millis():,}ms)"

In [ ]:
def build_aggregated_call_graph(file_path, symbols):
    """
    Reads a trace file and builds an aggregated call graph.
    """

    system_method = Method("-1", "system")
    current_method = system_method
    method_stack = []

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith(">"):
                method_id = line[2:]
                # Build a new method object if we do not have one yet for this ID
                if method_id not in current_method.calls_to.keys():
                    called_method = Method(method_id, symbols.get(method_id, method_id))
                else:
                    called_method = current_method.calls_to[method_id]
                # record the method call ...
                current_method.add_call(called_method)
                # ... and "enter" the method
                method_stack.append(current_method)
                current_method = called_method
            elif line.startswith("<"):
                method_id, duration = line[2:].split(';')
                # remember how long the method ran ...
                current_method.durations.append(int(duration))
                # ... and "exit" the method
                current_method = method_stack.pop()
            else:
                print(f"Warning: Skipping invalid line: {line}")

    return system_method


In [ ]:
def color_percentage(hex_color, percentage):
    """
    Calculates the hex color code for a given percentage of a base color.

    Args:
        hex_color (str): The base color in hex format (e.g., "#FF0000").
        percentage (int): The desired percentage (0-100).

    Returns:
        str: The hex color code for the calculated percentage.
    """

    if percentage < 0:
        #print(f"perctage clamped from {percentage} to 0")
        percentage = 0
    if percentage > 100:
        #print(f"perctage clamped from {percentage} to 100")
        percentage = 100

    return f"{hex_color}{hex(int(255 * percentage / 100))[2:].zfill(2)}"

In [ ]:
def print_dot(method, overall):
    # print node
    node_color = color_percentage("#FF0000", method.millis() / overall * 100)
    print(f"\"{method}\"[fillcolor=\"{node_color}\"]")
    
    # print called methods
    for called_method in method.calls_to.values():
        # recursivly print called method
        print_dot(called_method, overall)
        #edge_color = color_percentage("#FF0000", called_method.duration() / overall * 100)
        # then print edge to called method
        print(f"\"{method}\" -> \"{called_method}\"[label=\"{called_method.nCalled:,}\"]") #, fillcolor=\"{edge_color}\", color=\"{node_color}\"]")

In [ ]:
def print_dot_graph(method, overall):
    print("digraph G { node[shape=box,style=filled];")
    print_dot(method, overall)
    print("}")

In [ ]:
# Example usage:
file_path = r"C:\path\to\trace.txt"
symbols_path = file_path.replace("trace", "symbols")
with open(symbols_path, 'r') as f:
    symbols = json.load(f)
system_node = build_aggregated_call_graph(file_path, symbols)

In [ ]:
main = next(iter(system_node.calls_to.values()))
overall = main.millis()
# Copy this DOT output 
print_dot_graph(main, overall)